In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# define the model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 2*input_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(2*input_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

def initialize_population(population_size, num_features):
    return torch.randint(2, (population_size, num_features), dtype=torch.float32)

def evaluate_population(population, train_data, test_data, num_features, hidden_size, output_size, epochs=10):
    fitness_scores = []
    for chromosome in population:
        selected_features = torch.nonzero(chromosome).squeeze().tolist()
        if not selected_features:
            # prevent the case that none of the features is selected
            fitness_scores.append(0)
            continue

        # select features
        train_features = train_data[:, selected_features]
        test_features = test_data[:, selected_features]

        # train
        if isinstance(selected_features,int):
            MLP(1,hidden_size,output_size)
        else:
            model = MLP(len(selected_features), hidden_size, output_size)
        criterion = nn.L1Loss()
        optimizer = optim.Adam(model.parameters(), lr=0.01)

        for epoch in range(epochs):
            outputs = model(train_features)
            loss = criterion(outputs, train_labels)
            l1_regularization = torch.tensor(0.0, requires_grad = True)
            for param in model.parameters():
                l1_regularization += torch.norm(param, p=1)
            loss += l1_regularization * 0.005
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # test the selected feature
        with torch.no_grad():
            test_outputs = model(test_features)
            _, predicted = torch.max(test_outputs.data, 1)
            accuracy = (predicted == test_labels).sum().item() / len(test_labels)
            fitness_scores.append(accuracy)

    return np.array(fitness_scores)


def select_parents(population, fitness_scores):
    population_np = population.numpy()

    sorted_indices = np.argsort(fitness_scores)[::-1]
    selected_indices = sorted_indices[:2] 

    selected_parents = torch.from_numpy(population_np[selected_indices])

    return selected_parents

def crossover(parents):
    crossover_point = torch.randint(1, parents.shape[1], (1,))
    child1 = torch.cat((parents[0, :crossover_point], parents[1, crossover_point:]))
    child2 = torch.cat((parents[1, :crossover_point], parents[0, crossover_point:]))
    return child1, child2

def mutate(child, mutation_rate):
    mutation_mask = (torch.rand(child.shape) < mutation_rate).float()
    child = (child + mutation_mask) % 2
    return child

# main loop
def genetic_algorithm(train_data, test_data, num_features, hidden_size, output_size, population_size=50, generations=50, mutation_rate=0.01):
    population = initialize_population(population_size, num_features)

    for generation in range(generations):
        fitness_scores = evaluate_population(population, train_data, test_data, num_features, hidden_size, output_size)
        parents = select_parents(population, fitness_scores)

        new_population = []
        for _ in range(population_size // 2):
            child1, child2 = crossover(parents)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            new_population.extend([child1, child2])

        population = torch.stack(new_population)

    best_individual_index = np.argmax(fitness_scores)
    best_individual = population[best_individual_index]

    return best_individual


train_data = torch.from_numpy(np.load("../data/features_rand_train.npy")).float()
test_data = torch.from_numpy(np.load("../data/features_rand_test.npy")).float()
train_labels = torch.from_numpy(np.load("../data/simu_20000_0.1_90_140_train.npy")[:,1004]).long().reshape(20000,1)
test_labels = torch.from_numpy(np.load("../data/simu_10000_0.1_141_178_test.npy")[:,1004]).long().reshape(10000,1)
# print(test_labels.shape)


best_individual = genetic_algorithm(train_data, test_data, num_features=11, hidden_size=8, output_size=1)

# Output the selection result
print("Best Individual:", best_individual)

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.